## Deep Convolutional Generative Adversarial Networks

### About DCGANs...
- Published as a conference paper at ICLR 2016 by Alec Radford et al. (Alec Radford later joined OpenAI where he pioneered the works on GPTs!)
- The original paper **Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks** can be found at <https://arxiv.org/abs/1511.06434>
- One of the first successful applications of CNN architecture in the unsupervised learning paradigm
- Propose a set of guidelines on the architectural topology for GANs based on CNNs
- The authors show that the generator possess interesting vector arithmetic properties 

### Importing the necessary libraries

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from model import Discriminator, Generator, initialize_weights
import matplotlib.pyplot as plt

### Architectural guidelines:
- Remove all fully-connected hidden layers
- Use batchnorm in both the Generator and the Discriminator (except at the output of G and the input of D)
- Use ReLU activation in the Generator (except at the output layer that uses tanh)
- Use LeakyReLU activation in the Discriminator with a slope of 0.2

Now, look at **model.py** for the implementation of G and D along with the weight initialization routine

### Hyperparameter configuration (following the paper)

In [ ]:
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 128
IMAGE_SIZE = 64
IMG_CHANNELS = 3
NOISE_DIM = 100
NUM_EPOCHS = 5
DISC_FEAT = 64
GEN_FEAT = 64
BETA1 = 0.5
MANUAL_SEED = 42

### For reproducibility...

In [ ]:
torch.manual_seed(MANUAL_SEED)
random.seed(MANUAL_SEED)
torch.use_deterministic_algorithms(mode=True)

### Dataset loading / preprocessing

- To see that this architecture could generate "good-looking" RGB images, we work with the CelebA dataset <https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html>. 
- To download just the training images, just use this kaggle link: <https://www.kaggle.com/datasets/504743cb487a5aed565ce14238c6343b7d650ffd28c071f03f2fd9b25819e6c9> and extract the files to a folder named **celeb_dataset** in your root (contains ~202K images of celebrities)

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize( (0.5,0.5,0.5), (0.5,0.5,0.5)),
    ]
)
# For MNIST...
# dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms, download=True) # set IMG_CHANNELS = 1

# For CelebA
dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
real_batch = next(iter(dataloader))

plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(dev)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

### Initializing the models...

In [ ]:
G = Generator(NOISE_DIM, IMG_CHANNELS, GEN_FEAT).to(dev)
G.apply(initialize_weights)
D = Discriminator(IMG_CHANNELS, DISC_FEAT).to(dev)
D.apply(initialize_weights)


### Loss function and optimizer

In [ ]:
opt_G = optim.Adam(G.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))
opt_D = optim.Adam(D.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))
criterion = nn.BCELoss()

### Config for Tensorboard

In [ ]:
fixed_z = torch.randn(64, NOISE_DIM, 1, 1).to(dev)
writer_data = SummaryWriter(f"logs/CelebA")
writer_fake = SummaryWriter(f"logs/Fake")
step = 0

### Training the GAN...

In [ ]:
# Setting the Generator and Discriminator to training mode...
G.train()
D.train()

for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, _) in enumerate(dataloader):
        D.zero_grad()
        real = real.to(dev)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        # Train with all-real batch
        disc_real = D(real).reshape(-1)
        loss_D_real = criterion(disc_real, torch.ones_like(disc_real))
        loss_D_real.backward()
        D_x = disc_real.mean().item()

        # Train with all-fake batch
        z = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(dev)
        fake = G(z)
        disc_fake = D(fake.detach()).reshape(-1)
        loss_D_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_D_fake.backward()
        D_G_z1 = disc_fake.mean().item()

        loss_D = loss_D_real + loss_D_fake 
        
        opt_D.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        G.zero_grad()
        output = D(fake).reshape(-1)
        loss_G = criterion(output, torch.ones_like(output))
        
        loss_G.backward()
        D_G_z2 = output.mean().item()
        opt_G.step()

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \n
                Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}, D(x): {D_x}, D(G(z)): {D_G_z1} / {D_G_z2}"
            )

            with torch.no_grad():
                fake = G(fixed_z)
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                writer_data.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1